Gia Thuận

----

# Import thư viện

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# Thu thập dữ liệu

Dữ liệu được sử dụng trong đồ án này là tập dữ liệu được thu thập sẵn và được đăng tải trên Kaggle.

Nguồn dữ liệu: [Loan Application Data](https://www.kaggle.com/vipin20/loan-application-data?fbclid=IwAR1fjcdLGjY-5E5rgJMjQwrH2PGrnum4G3dyXH2ZnEIE8rg1RXMu-7hA95Q)

Thông tin mô tả dữ liệu được trích dẫn một phần từ nguồn trên:
>About data set
Among all industries, Banking domain has the largest use of analytics & data science methods. This data set would provide you enough taste of working on data sets from insurance companies and banks, what challenges are faced, what strategies are used, etc. This is a classification problem. The data has 615 rows and 14 features to predict weather loan approved or not approved.

>Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have given a problem to identify the customers segments, those are eligible for loan amount so that they can specifically target these customers.



# Khám phá dữ liệu

## Đọc dữ liệu từ file vào dataframe

In [2]:
loan_df = pd.read_csv('Data/df1_loan.csv', delimiter=',', index_col=0)
loan_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,$5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,$6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,$3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,$4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,$6000.0


## Dữ liệu bao gồm bao nhiêu dòng và bao nhiêu cột?

In [3]:
loan_df.shape

(500, 14)

## Mỗi dòng có ý nghĩa gì?


Mỗi dòng là thông tin về một đơn đăng ký khoản vay của một người nào đó.

## Có dòng nào bị lặp không? Nếu có thì bao nhiêu dòng bị lặp?

In [4]:
re = loan_df.duplicated()
if re.any():
    print(f"Số dòng bị lặp: {sum(re)}")
else:
    print('Không dòng nào bị lặp')

Không dòng nào bị lặp


## Mỗi cột có ý nghĩa gì?

|Tên cột|Ý nghĩa|
|:---|:---:|
|**Loan_ID**|Mã định danh của đơn đăng ký vay tiền (mỗi Loan_ID là duy nhất)|
|**Gender**|Giới tính của người nộp đơn đăng ký vay tiền|
|**Married**|Tình trạng hôn nhân của người nộp đơn đăng ký vay tiền|
|**Dependents**|Số người phụ thuộc vào người nộp đơn|
|**Education**|Trình độ vấn của người nộp đơn đăng ký vay tiền|
|**Self_Employed**|Người nộp đơn có đang làm việc tự do hay không|
|**ApplicantIncome**|Thu nhập của người nộp đơn đăng ký vay tiền|
|**CoapplicantIncome**|Thu nhập của người đồng nộp đơn(người có cùng trách nhiệm với người nộp đơn)|
|**LoanAmount**|Số tiền cho vay|
|**Loan_Amount_Term**|Thời hạn cho vay tiền|
|**Credit_History**|Lịch sử tín dụng|
|**Property_Area**|Nhà ở nằm ở khu vực nào|
|**Loan_Status**|Trạng thái khoản vay (có được duyệt hay không)|
|**Total_Income**|Tổng thu nhập (bao gồm tổng thu nhập của người nộp đơn và người cùng chịu trách nhiệm)|





## Kiểu dữ liệu của mỗi cột là gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần tiền xử lý không?

In [5]:
loan_df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
Total_Income          object
dtype: object

**Nhận xét**: 
- Cột **ApplicantIncome** có kiểu dữ liệu là int64, trong khi các cột numerical khác lại có kiểu dữ liệu là float64. Vậy để dễ dàng tính toán, ta chuyển cột ApplicantIncome sáng kiểu float.

- Cột **Total_Income** chưa có kiểu dữ liệu số, đặc biệt hơn là có dấu $ trước mỗi giá trị. Cần loại bỏ dấu đó  và  chuyển về kiểu dữ liệu float.

- Cột **Dependents** có những giá trị 3+, ta thống nhất chuyển những giá trị đó thành 4, đồng thời chuyển cả cột về kiểu dữ liệu float.

- Cột **Loan_status** có 2 giá Y và N, cần chuẩn hóa thành Yes và No giúp cho người đọc dễ hiểu và đồng bộ kiểu giá trị với cột Married (có cùng kiểu giá trị).

## Tiền xử lý

### Cột ApplicantIncome

In [6]:
loan_df['ApplicantIncome'] = loan_df['ApplicantIncome'].astype('float')

### Cột Total_Income

In [7]:
loan_df['Total_Income'] = loan_df['Total_Income'].str.replace('$','',regex=True).astype('float')

### Cột Dependents

Các giá trị đặc trưng của cột:

In [8]:
loan_df['Dependents'].unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [9]:
pattern_convert = {'3+':'4'}
loan_df['Dependents'] = loan_df['Dependents'].apply(\
                            lambda x: x if x not in pattern_convert else pattern_convert[x]).astype('float')

### Cột Loan_Status

In [10]:
pattern_convert = {'Y':'Yes',
                  'N': 'No'}
loan_df['Loan_Status'] = loan_df['Loan_Status'].apply(lambda x: pattern_convert[x])

### Kiểm tra lại

In [11]:
loan_df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents           float64
Education             object
Self_Employed         object
ApplicantIncome      float64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
Total_Income         float64
dtype: object

Khá ổn rồi....

## Với mỗi cột dữ liệu dạng numerical, các giá trị được phân bố như thế nào?

Ta tiến hành tính các thông số missing_ratio (tỉ lệ phần trăm các giá trị thiếu), min, median, max.


In [12]:
def missing_ratio(s):
    return s.isna().mean()*100

num_cols = loan_df.select_dtypes(include=['float32','float64','int32','int64'])
num_cols_info = num_cols.agg([missing_ratio,'min','median','max'])
num_cols_info

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income
missing_ratio,2.4,0.0,0.0,3.6,2.8,8.2,0.0
min,0.0,150.0,0.0,17.0,12.0,0.0,1442.0
median,0.0,3854.0,1125.5,126.5,360.0,1.0,5332.5
max,4.0,81000.0,20000.0,700.0,480.0,1.0,81000.0


## Với mỗi cột dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Ta tính cần tính
- missing_ratio: tỉ lệ phần trăm các giá trị thiếu.
- nums_diff_values: số lượng giá trị khác nhau (không xét giá trị thiếu).
- diff_vals_ratio: tỉ lệ phần trăm của mỗi giá trị khác nhau ứng với mỗi cột (không xét giá trị thiếu), lưu dạng dictionary với key là tên giá trị và value là tỉ lệ phần trăm của giá trị đó.

In [13]:
def nums_diff_values(s):
    return s.dropna().nunique()
def diff_vals_ratio(s):
    s = s.dropna()
    return (s.value_counts()/len(s)*100).to_dict()

cate_cols = loan_df.select_dtypes(exclude=['float32','float64','int32','int64'])
cate_cols_info = cate_cols.agg([missing_ratio, nums_diff_values, diff_vals_ratio])
cate_cols_info

,Loan_ID,Gender,Married,Education,Self_Employed,Property_Area,Loan_Status
missing_ratio,0.0,1.8,0.6,0.0,5.4,0.0,0.0
nums_diff_values,500,2,2,2,2,3,2
diff_vals_ratio,"{'LP001199': 0.2, 'LP001052': 0.2, 'LP001119':...","{'Male': 81.4663951120163, 'Female': 18.533604...","{'Yes': 64.7887323943662, 'No': 35.2112676056338}","{'Graduate': 78.60000000000001, 'Not Graduate'...","{'No': 86.04651162790698, 'Yes': 13.9534883720...","{'Semiurban': 37.6, 'Urban': 34.2, 'Rural': 28...","{'Yes': 69.0, 'No': 31.0}"


Ta thấy các dữ liệu dạng categorical khá giống với mô tả về tập dữ liệu.